**Text Classification with ArabicTransformer and TPU with Keras API**

*   First, you need to activate TPU by going to Runtime-> Change RunTime Type -> TPU .

*   This example was tested with HuggingFace Transformer Library version v4.11.2 . If you experience any issue roll back to this version.

*   This example uses Keras API with TensorFlow. Keras API makes it easy for fine-tuning since you don't need to have Google Bucket like the case in native TensorFlow with TPU. 

*   In our paper, we use the original implementation of funnel transformer (PyTorch) (https://github.com/laiguokun/Funnel-Transformer) and V100 GPU, which is no longer provided for Google Colab Pro users. We will update you later on our modified code of the Funnel Transfomer library. However, in the meantime, you need to find the best hyperparameters here and don't rely on our setting in this notebook since the implementation is different from our paper. However, our current set of hyperparameters in this example is still close to what we reported in our paper. You may also get better results with our model than what we reported if you extend the grid search (:

* You can easily run this code on GPU since TensorFlow and Keras API can work with GPU.

* One disadvantage of this approach over PyTorchXLA, which we show in other examples, is the instability of the result for each run. If you run the same code with the same hyperparameters with the same seed, you may get different results. This is because the seed in TensorFlow is related to the graph itself. Try to solve this problem from your own side.


*   This example is based on the GLUE fine-tuning task example from the hugging face team, but it can work with any text classification task and can be used to fine-tune any Arabic Language Model that was uploaded to HuggingFace Hub here https://huggingface.co/models . A text classification task is where we have a sentence and a label like sentiment analysis tasks. You just need to name the header of the first sentence that you need to classify to sentence1 and label to "label" column. If you want to classify two sentences, then name the first sentence as sentence1 and the other one to sentence2.


*   We did not include language models that use pre-segmentation (FARASA), such as AraBERTv2, in the list of models below. You can do the pre-segmentation part from your own side using codes that AUB Mind published here https://github.com/aub-mind/arabert. Then use our code to fine-tune AraBERTv2 or similar models.

*   If the model scale is changed (small, base, large) or the architecture is different (Funnel, BERT, ELECTRA, ALBERT), you need to change your hyperparameters. Evaluating all models using the same hyperparameters across different scales and architectures is bad practice to report results.

*   We suggest you use this example to find the best hyperparameters and then use PyTorch XLA to report the result with the same hyperparameters that you find the best. This will lead to more reproducibility for your work. See these posts :

https://stackoverflow.com/questions/32419510/how-to-get-reproducible-results-in-keras

https://stackoverflow.com/questions/50659482/why-cant-i-get-reproducible-results-in-keras-even-though-i-set-the-random-seeds

In [1]:
!git clone https://github.com/huggingface/transformers
!pip3 install -e transformers 
!pip3 install -r transformers/examples/tensorflow/text-classification/requirements.txt

Cloning into 'transformers'...
remote: Enumerating objects: 85569, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 85569 (delta 8), reused 17 (delta 3), pack-reused 85541
Receiving objects: 100% (85569/85569), 68.50 MiB | 34.76 MiB/s, done.
Resolving deltas: 100% (61486/61486), done.
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 74.4 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 71.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop for transformers
     |████████████████████████████████| 270 kB 5.2 MB/s 
    

In [2]:
import pandas as pd
!rm -r /content/data
!mkdir -p data/raw/scarcasmv2
!mkdir -p data/scarcasmv2
!wget -O data/raw/scarcasmv2/dev.csv https://raw.githubusercontent.com/iabufarha/ArSarcasm-v2/main/ArSarcasm-v2/testing_data.csv
!wget -O data/raw/scarcasmv2/train.csv https://raw.githubusercontent.com/iabufarha/ArSarcasm-v2/main/ArSarcasm-v2/training_data.csv
df = pd.read_csv(r'data/raw/scarcasmv2/train.csv', header=0,escapechar='\n',usecols = [0,2],names=["sentence1", "label"])
df.to_csv('data/scarcasmv2/train.csv',index=False)
df.to_csv('data/scarcasmv2/train.tsv',sep='\t',index=False)
df = pd.read_csv(r'data/raw/scarcasmv2/dev.csv', header=0, escapechar='\n',usecols = [0,2],names=["sentence1", "label"])
df.to_csv('data/scarcasmv2/dev.csv',index=False)
df.to_csv('data/scarcasmv2/dev.tsv',sep='\t',index=False)

rm: cannot remove '/content/data': No such file or directory
--2021-10-02 13:48:15--  https://raw.githubusercontent.com/iabufarha/ArSarcasm-v2/main/ArSarcasm-v2/testing_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 585081 (571K) [text/plain]
Saving to: ‘data/raw/scarcasmv2/dev.csv’

data/raw/scarcasmv2 100%[===================>] 571.37K  --.-KB/s    in 0.04s   

2021-10-02 13:48:15 (13.6 MB/s) - ‘data/raw/scarcasmv2/dev.csv’ saved [585081/585081]

--2021-10-02 13:48:16--  https://raw.githubusercontent.com/iabufarha/ArSarcasm-v2/main/ArSarcasm-v2/training_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.co

We need to add this line to TensorFlow code can accept pre-trained models that are in format of HuggingFace PyTorch

In [3]:
!sed -i '420i\\t          from_pt=True,' /content/transformers/examples/tensorflow/text-classification/run_text_classification.py

In [4]:
import pandas as pd
from sklearn.metrics import f1_score,classification_report,accuracy_score
def calc_scarcasm(y_pred,y_true):
  y_pred=pd.read_csv(y_pred, sep='\t',header=None,usecols=[1] )
  y_true=pd.read_csv(y_true,usecols=[1],header=None)
  print("Accur Score:",accuracy_score(y_true, y_pred)*100)
  print("F1 PN Score:",f1_score(y_true, y_pred,labels=['NEG','POS'],average="macro")*100)
  print("########################### Full Report ###########################")
  print(classification_report(y_true, y_pred,digits=4,labels=['NEG','POS'] ))

In [14]:
import os
model= "sultan/ArabicTransformer-small" #@param ["sultan/ArabicTransformer-small","sultan/ArabicTransformer-intermediate","sultan/ArabicTransformer-large","aubmindlab/araelectra-base-discriminator","asafaya/bert-base-arabic","aubmindlab/bert-base-arabertv02","kuisailab/albert-base-arabic","aubmindlab/bert-large-arabertv02"]
task= "scarcasmv2" #@param ["scarcasmv2"]
batch_size = 64 #@param {type:"slider", min:4, max:128, step:4}
learning_rate = "3e-5"#@param ["1e-4","2e-4", "3e-4", "1e-5","2e-5","3e-5","4e-5","5e-5","6e-5","7e-5","8e-5","9e-5"]
epochs_num = 2 #@param {type:"slider", min:1, max:50, step:1}
max_seq_length= "256" #@param ["128", "256", "384","512"]
os.environ['batch_size'] = str(batch_size)
os.environ['learning_rate'] = str(learning_rate)
os.environ['epochs_num'] = str(epochs_num)
os.environ['task'] = str(task)
os.environ['model'] = str(model)
os.environ['max_seq_length'] = str(max_seq_length)
!python transformers/examples/tensorflow/text-classification/run_text_classification.py --model_name_or_path $model \
--train_file data/$task/train.csv \
--test_file data/$task/dev.csv \
--output_dir output_dir/$task \
--overwrite_cache \
--overwrite_output_dir \
--logging_steps 1000000 \
--max_seq_length $max_seq_length \
--per_device_train_batch_size $batch_size \
--learning_rate $learning_rate \
--warmup_ratio 0.1 \
--num_train_epochs $epochs_num \
--save_steps 50000 \
--do_train \
--do_predict

10/02/2021 14:02:37 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=output_dir/scarcasmv

In [15]:
calc_scarcasm('/content/output_dir/scarcasmv2/test_results.txt','/content/data/scarcasmv2/dev.csv')

Accur Score: 70.10996334555148
F1 PN Score: 72.54026732668572
########################### Full Report ###########################
              precision    recall  f1-score   support

         NEG     0.7656    0.8259    0.7946      1677
         POS     0.6252    0.6904    0.6562       575

   micro avg     0.7291    0.7913    0.7589      2252
   macro avg     0.6954    0.7582    0.7254      2252
weighted avg     0.7298    0.7913    0.7593      2252

